In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [4]:
filepath = "file:///home/talentum/test-jupyter/P2/M5/p2/constitution.txt"
df = spark.read.text(filepath)

In [5]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
df.show(3)

+--------------------+
|               value|
+--------------------+
|We the People of ...|
|Union, establish ...|
|common defence, p...|
+--------------------+
only showing top 3 rows



In [7]:
df.describe()

DataFrame[summary: string, value: string]

In [8]:
type(df)

pyspark.sql.dataframe.DataFrame

In [9]:
df.count()

649

#### create a dataframe. With 2 columns, 1 column having all the unique words and 2nd column with frequency of the words

In [10]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
df = df.withColumn("split", F.split(df['value'], "\s+"))

In [12]:
df.printSchema()

root
 |-- value: string (nullable = true)
 |-- split: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
df.show(3)

+--------------------+--------------------+
|               value|               split|
+--------------------+--------------------+
|We the People of ...|[We, the, People,...|
|Union, establish ...|[Union,, establis...|
|common defence, p...|[common, defence,...|
+--------------------+--------------------+
only showing top 3 rows



In [15]:
df = df.select(F.explode(df["split"]))

In [16]:
df.count()

8268

In [17]:
df.show()

+---------+
|      col|
+---------+
|       We|
|      the|
|   People|
|       of|
|      the|
|   United|
|  States,|
|       in|
|    Order|
|       to|
|     form|
|        a|
|     more|
|  perfect|
|         |
|   Union,|
|establish|
| Justice,|
|   insure|
| domestic|
+---------+
only showing top 20 rows



In [18]:
df.printSchema()

root
 |-- col: string (nullable = true)



In [20]:
df = df.groupBy("col").count()

In [22]:
type(df)

pyspark.sql.dataframe.DataFrame

In [23]:
df.printSchema()

root
 |-- col: string (nullable = true)
 |-- count: long (nullable = false)



In [27]:
df.show(3)

+------------+-----+
|         col|count|
+------------+-----+
|  Inhabitant|    3|
|       those|    6|
|Affirmation.|    1|
+------------+-----+
only showing top 3 rows



In [29]:
df = df.sort("count", ascending = False)

In [31]:
df.show(5)

+-----+-----+
|  col|count|
+-----+-----+
|  the|  662|
|     |  645|
|   of|  493|
|shall|  293|
|  and|  256|
+-----+-----+
only showing top 5 rows

